### Model 3 Implementation (Method 1)
Model is constructed with multiple outs, calculated loss between outputs is independent from each other

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random, json
import cv2
import tensorflow as tf
import seaborn as sns
import time
from keras.callbacks import TensorBoard
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_sample_image

### Preparing the Dataset

In [3]:
# Logging stuff
model_name  = "cnnMask_resnet_multiout_default_loss_{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(model_name))

In [4]:
BATCH_SIZE = 5
NUM_EPOCHS = 15
IMG_SIZE   = (128, 128)

percent_val = 0.1

mask_train = pd.read_csv('../FINAL_DATASET/traindf.csv')
mask_test  = pd.read_csv('../FINAL_DATASET/test.csv')

mask_train['classname'].value_counts(), mask_test['classname'].value_counts(),

(face_with_mask              2034
 face_no_mask                 773
 face_other_covering          675
 face_with_mask_incorrect      68
 Name: classname, dtype: int64,
 face_with_mask              881
 face_no_mask                325
 face_other_covering         284
 face_with_mask_incorrect     32
 Name: classname, dtype: int64)

In [5]:
equal_classes = pd.concat([mask_train, mask_test]).groupby('classname')\
# Uncomment to make number of images from each class equal
# num_per_class = equal_classes.size().max() # or equal_classes.size().max(), equal_classes.size().min()
# equal_classes = pd.DataFrame(equal_classes.apply(lambda x : x.sample(num_per_class, replace = True)).reset_index(drop = True))
equal_classes = equal_classes.sample(frac = 1)

equal_classes['classname'].value_counts()

face_no_mask                2915
face_with_mask_incorrect    2915
face_with_mask              2915
face_other_covering         2915
Name: classname, dtype: int64

In [6]:
equal_classes['type' ] = equal_classes['classname']
equal_classes['usage'] = equal_classes['classname']

replacement_dict = {
    'type': { 
        'face_other_covering'       : 0.0,
        'face_with_mask_incorrect'  : 1.0,
        'face_with_mask'            : 1.0,
        'face_no_mask'              : 0.0,
    },  
    'usage': { 
        'face_other_covering'       : 0.0,
        'face_with_mask_incorrect'  : 0.0,
        'face_with_mask'            : 1.0,
        'face_no_mask'              : 0.0,
    }
}

equal_classes = equal_classes.replace(replacement_dict)
equal_classes[['type', 'usage']].value_counts()

mask_train, mask_test = train_test_split(equal_classes, test_size = percent_val, stratify = equal_classes[['type', 'usage']])
valid_mask_test       = mask_test[mask_test['type'] != 0.0]

(mask_train[['type', 'usage']].value_counts(), mask_test[['type', 'usage']].value_counts(), valid_mask_test[['type', 'usage']].value_counts())

(type  usage
 0.0   0.0      5247
 1.0   1.0      2624
       0.0      2623
 dtype: int64,
 type  usage
 0.0   0.0      583
 1.0   0.0      292
       1.0      291
 dtype: int64,
 type  usage
 1.0   0.0      292
       1.0      291
 dtype: int64)

In [7]:
def adjust_image(input_image):
    brightness = random.choice([1.0, 0.8, 1.2])
    contrast   = random.choice([1.0, 0.8, 1.2])
    saturation = random.choice([1.0, 0.8, 1.2])

    img_proc = cv2.cvtColor(input_image, cv2.COLOR_RGB2HSV)
    np.multiply(img_proc, np.array([ 1.0, saturation, 1.0 ], dtype = np.single), out = img_proc)
    
    img_proc[img_proc > 255] = 255
    img_proc[img_proc < 0]   = 0

    cv2.cvtColor(img_proc, cv2.COLOR_HSV2RGB, dst = img_proc)
    np.multiply(img_proc, brightness, out = img_proc)
    np.add(img_proc, ((1-contrast) * 100))

    img_proc[img_proc > 255] = 255
    img_proc[img_proc < 0]   = 0
    img_proc  = img_proc.astype(np.float32) * (1.0 / 255)

    return img_proc

image_gen  = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range      = 0.1, 
                                                             height_shift_range     = 0.1, 
                                                             horizontal_flip        = True,
                                                             preprocessing_function = adjust_image)

In [8]:
train_ds = image_gen.flow_from_dataframe(mask_train, '../FINAL_DATASET/croppedv2', 
                                              x_col       = 'newFilename', 
                                              y_col       = ['type', 'usage'], 
                                              target_size = IMG_SIZE, 
                                              class_mode  = 'multi_output',
                                              subset      = "training", 
                                              batch_size  = BATCH_SIZE)

test_ds = image_gen.flow_from_dataframe(mask_test, '../FINAL_DATASET/croppedv2', 
                                            x_col       = 'newFilename', 
                                            y_col       = ['type', 'usage'], 
                                            target_size = IMG_SIZE, 
                                            class_mode  = 'multi_output',
                                            batch_size  = BATCH_SIZE)

valid_mask_test_ds = image_gen.flow_from_dataframe(valid_mask_test, '../FINAL_DATASET/croppedv2', 
                                            x_col       = 'newFilename', 
                                            y_col       = ['type', 'usage'], 
                                            target_size = IMG_SIZE, 
                                            class_mode  = 'multi_output',
                                            batch_size  = BATCH_SIZE)                                            

Found 10494 validated image filenames.
Found 1166 validated image filenames.
Found 583 validated image filenames.


### Usage and Type classifier

In [9]:
input_img  = tf.keras.layers.Input(shape = (128, 128, 3))

def build_cnn_model():
    base = tf.keras.applications.ResNet50(include_top = False, input_shape = (*IMG_SIZE,3), pooling = 'max')(input_img)
    base = tf.keras.layers.Flatten()(base)
    base = tf.keras.layers.Dense(256, activation = 'relu')(base)
    
    type_out = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'mask_type')(base)

    usage_out = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'mask_usage')(base)

    model = tf.keras.Model(inputs = input_img, outputs = [type_out, usage_out])
    return model

model = build_cnn_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 2048)         23587712    ['input_1[0][0]']                
                                                                                                  
 flatten (Flatten)              (None, 2048)         0           ['resnet50[0][0]']               
                                                                                                  
 dense (Dense)                  (None, 256)          524544      ['flatten[0][0]']            

In [10]:
model.compile(optimizer = 'adam', 
              loss      = [tf.keras.losses.BinaryCrossentropy(), tf.keras.losses.BinaryCrossentropy()], 
              metrics   = [ 
                    'accuracy', 
                    tf.keras.metrics.Precision(), 
                    tf.keras.metrics.Recall(),
                    tf.keras.metrics.TruePositives(),
                    tf.keras.metrics.FalsePositives(),
                    tf.keras.metrics.TrueNegatives(),
                    tf.keras.metrics.FalseNegatives(),
                ])

In [11]:
history = model.fit(train_ds, validation_data = test_ds, epochs = NUM_EPOCHS, callbacks = [tensorboard])

Epoch 1/15
2099/2099 [==============================] - 174s 78ms/step - loss: 1.0997 - mask_type_loss: 0.5499 - mask_usage_loss: 0.5498 - mask_type_accuracy: 0.7732 - mask_type_precision: 0.7715 - mask_type_recall: 0.7763 - mask_type_true_positives: 4073.0000 - mask_type_false_positives: 1206.0000 - mask_type_true_negatives: 4041.0000 - mask_type_false_negatives: 1174.0000 - mask_usage_accuracy: 0.7676 - mask_usage_precision: 0.5569 - mask_usage_recall: 0.3453 - mask_usage_true_positives: 906.0000 - mask_usage_false_positives: 721.0000 - mask_usage_true_negatives: 7149.0000 - mask_usage_false_negatives: 1718.0000 - val_loss: 3.7003 - val_mask_type_loss: 2.4918 - val_mask_usage_loss: 1.2085 - val_mask_type_accuracy: 0.5583 - val_mask_type_precision: 0.5311 - val_mask_type_recall: 0.9949 - val_mask_type_true_positives: 580.0000 - val_mask_type_false_positives: 512.0000 - val_mask_type_true_negatives: 71.0000 - val_mask_type_false_negatives: 3.0000 - val_mask_usage_accuracy: 0.3722 - val

In [12]:
results = model.evaluate(test_ds, callbacks = [tensorboard])

234/234 [==============================] - 22s 92ms/step - loss: 0.6506 - mask_type_loss: 0.3859 - mask_usage_loss: 0.2646 - mask_type_accuracy: 0.8370 - mask_type_precision: 0.9712 - mask_type_recall: 0.6947 - mask_type_true_positives: 405.0000 - mask_type_false_positives: 12.0000 - mask_type_true_negatives: 571.0000 - mask_type_false_negatives: 178.0000 - mask_usage_accuracy: 0.8868 - mask_usage_precision: 0.8502 - mask_usage_recall: 0.6632 - mask_usage_true_positives: 193.0000 - mask_usage_false_positives: 34.0000 - mask_usage_true_negatives: 841.0000 - mask_usage_false_negatives: 98.0000


In [13]:
results_for_valid_masks = model.evaluate(valid_mask_test_ds, callbacks = [tensorboard])

117/117 [==============================] - 12s 99ms/step - loss: 1.0527 - mask_type_loss: 0.6181 - mask_usage_loss: 0.4346 - mask_type_accuracy: 0.7084 - mask_type_precision: 1.0000 - mask_type_recall: 0.7084 - mask_type_true_positives: 413.0000 - mask_type_false_positives: 0.0000e+00 - mask_type_true_negatives: 0.0000e+00 - mask_type_false_negatives: 170.0000 - mask_usage_accuracy: 0.8148 - mask_usage_precision: 0.9067 - mask_usage_recall: 0.7010 - mask_usage_true_positives: 204.0000 - mask_usage_false_positives: 21.0000 - mask_usage_true_negatives: 271.0000 - mask_usage_false_negatives: 87.0000


In [14]:
model.save('saved-models/{}'.format(model_name))

INFO:tensorflow:Assets written to: saved-models/cnnMask_resnet_multiout_default_loss_1650938061\assets
